In [165]:
import pandas as pd
import requests
import os

from tqdm.auto import tqdm  # for notebooks

tqdm.pandas()

In [166]:
file_path = "replays.parquet"
if os.path.exists(file_path):
    df_disk = pd.read_parquet(file_path)
    if "id" in df_disk.columns:
        df_disk = df_disk.set_index("id")


fetch_new = False
if fetch_new:
    apiUrl = "https://api.bar-rts.com/replays?limit=500&preset=team&hasBots=true"

    json = requests.get(apiUrl, headers={"User-Agent": "tetrisface"}).json()

    data = json["data"]

    df_api = pd.DataFrame.from_records(data).set_index("id")

    if df_disk is None:
        df_root = df_api
    else:
        df_root = pd.concat(
            [df_disk, df_api.loc[df_api.index.difference(df_disk.index)]],
            verify_integrity=True,
            axis=0,
        )
    df_root.startTime = pd.to_datetime(df_root.startTime)
    df_root.to_parquet(file_path)
else:
    df_root = df_disk

In [167]:
pd.options.mode.copy_on_write = True
import time


def is_raptors(row):
    for team in row:
        for ai in team["AIs"]:
            if ai["shortName"] == "RaptorsAI":
                return True
    return False


def is_draw(row):
    results = []
    for team in row:
        results.append(team["winningTeam"])
    return len(team) <= 1 or all(
        x == results[0] for x in [team["winningTeam"] for team in row]
    )


def winners(row):
    _winners = []
    for team in row:
        if team["winningTeam"] is True:
            if len(team["Players"]) > 0:
                _winners.extend([player["name"] for player in team["Players"]])
            elif len(team["AIs"]) > 0:
                _winners.extend([ai["shortName"] for ai in team["AIs"]])
    return _winners


def players(row):
    _players = []
    for team in row:
        _players.extend([player["name"] for player in team["Players"]])
    return _players


df_root_expanded = df_root.copy()
# print(f"root replays between {df_root_expanded["startTime"].min()[:10]} and {df_root_expanded["startTime"].max()[:10]}")

df_root_expanded["raptors"] = df_root_expanded["AllyTeams"].apply(is_raptors)
df_root_expanded["draw"] = df_root_expanded["AllyTeams"].apply(is_draw)
df_root_expanded["winners"] = df_root_expanded["AllyTeams"].apply(winners)
df_root_expanded["players"] = df_root_expanded["AllyTeams"].apply(players)

In [168]:
# from warnings import simplefilter

# simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

df_raptors = df_root_expanded[
    df_root_expanded["raptors"] & ~df_root_expanded["draw"]
]  # draws might be good to exclude

df_raptors["fetch_success"] = None


def api_game_settings(row):
    url = f"https://api.bar-rts.com/replays/{row.name}"
    response = requests.get(url, headers={"User-Agent": "tetrisface"})
    if response.status_code == 200:
        game_settings = response.json().get("gameSettings")
        for key, value in game_settings.items():
            # Add new column to DataFrame if the column doesn't exist
            if key not in df_raptors.columns:
                df_raptors[key] = None
            # Update DataFrame with fetched data
            row[key] = value
        time.sleep(1)
        row["fetch_success"] = True
    else:
        print(f"Failed to fetch data from {url}")
        time.sleep(1)
        row["fetch_success"] = False
    return row


# load in disk
if os.path.exists("replays_gamesettings.pickle"):
    df_disk = pd.read_pickle("replays_gamesettings.pickle")
    df_raptors.loc[df_disk.index, df_disk.columns] = df_disk

# fetch new
df_raptors_api = (
    df_raptors[df_raptors["fetch_success"].isnull()]
    .head(800)
    .progress_apply(
        api_game_settings,
        axis=1,
    )
)
df_raptors.loc[df_raptors_api.index, df_raptors_api.columns] = df_raptors_api

# store
df_raptors[df_raptors["fetch_success"].notnull()].to_pickle(
    "replays_gamesettings.pickle"
)

if len(df_raptors.loc[df_raptors.fetch_success == False]) > 0:
    print(f'failed to fetch {len(df_raptors[~df_raptors["fetch_success"]])} games')

100%|██████████| 709/709 [14:03<00:00,  1.19s/it]


In [169]:
from nuttyb import hp_multiplier_149, main

df_raptors["nuttyb_main"] = df_raptors.apply(
    lambda row: all(
        [
            tweak["value"] == row[tweak["location"]]
            for tweak in main
            if tweak["version"] == "1.48"
        ]
    )
    or all(
        [
            tweak["value"] == row[tweak["location"]]
            for tweak in main
            if tweak["version"] == "1.49"
        ]
    ),
    axis=1,
)


def nuttyb_difficulty(row):
    for _def in hp_multiplier_149:
        if row[_def["location"]] == _def["value"]:
            return _def["name"]
    return None


df_raptors["nuttyb_difficulty"] = df_raptors.apply(nuttyb_difficulty, axis=1)

df_raptors["difficulty"] = df_raptors.apply(
    lambda row: (
        "NuttyB " + row["nuttyb_difficulty"]
        if row["nuttyb_difficulty"] is not None
        else row["raptor_difficulty"]
    ),
    axis=1,
)

In [170]:
df_raptor_players = df_raptors.explode("players")
df_raptor_players.rename(columns={"players": "player"}, inplace=True)


df_raptor_players["won"] = df_raptor_players.apply(
    lambda row: row.player in row.winners, axis=1
)

In [178]:
print(
    f'replays between {df_raptor_players["startTime"].min()} and {df_raptor_players["startTime"].max()}'
)
df_raptor_players["difficulty"] = pd.Categorical(
    df_raptor_players["difficulty"],
    [
        "veryeasy",
        "easy",
        "normal",
        "hard",
        "veryhard",
        "epic",
        "epicer",
        "epicest",
        "NuttyB Epic",
        "NuttyB Epic+",
        "NuttyB Epic++",
        "NuttyB Epicer",
        "NuttyB Epicest",
    ],
)
total_unique = df_raptor_players.player.nunique()
agg_total = df_raptor_players.groupby(["difficulty"], observed=True).agg(
    {"player": ["nunique"]}
)
agg_won = (
    df_raptor_players[df_raptor_players["won"]]
    .groupby(["difficulty"], observed=True)
    .agg({"player": ["nunique"]})
)
joined = agg_total.join(agg_won, lsuffix="_tried", rsuffix="_won")
joined["% of trying players ever won"] = (
    joined.player_won["nunique"] / joined.player_total["nunique"] * 100
)
joined["% tried of total"] = joined.player_tried["nunique"] / total_unique * 100
joined["% won of total"] = joined.player_tried["nunique"] / total_unique * 100

pd.options.display.float_format = "{:,.0f}%".format
joined

SyntaxError: invalid syntax (3179479208.py, line 35)

In [172]:
# df_raptors = df_raptors.explode("players")
# df_raptors.rename(columns={"players": "player"}, inplace=True)
# df_raptors["won"] = df_raptors.apply(lambda row: row.player in row.winners, axis=1)

# df_raptors["week"] = pd.to_datetime(df_raptors["startTime"]).dt.month
# df = df_raptors.groupby(["player", "week"]).agg({"won": "mean", "id": "count"})
# df = df.rename(columns={"won": "winrate", "id": "replays"})

# df = df[df.replays > 4]

# rank_sum_column = "rank_sum"
# df[rank_sum_column] = (df.winrate.rank() * 0.1 + df.replays.rank() * 0.9) / len(df)
# df.sort_values(rank_sum_column, inplace=True, ascending=False)

# df[rank_sum_column] = round(df[rank_sum_column], 2).astype(str)

# df.winrate = df.winrate * 100

In [173]:
# pd.set_option("display.max_rows", 500)
# pd.options.display.float_format = "{:.0f}%".format

# for week, df_week in df.groupby("week"):
#     # df_week = df_week[df_week["replays"] > 4].reset_index()
#     df_week = df_week.reset_index()
#     df_week.index = np.arange(1, len(df_week) + 1)
#     print(df_week.head(20))
#     print()